In [288]:
# -*- coding: utf-8 -*-
from __future__ import division
import pickle
import requests

import jieba
import re
import operator

import eigen_config
from simplex import utils, logger
from simplex.model import KeyWordClassifier
from simplex.utils.finance_util import parse_report

class FinancePreprocessV2(object):
    '''
    财报文章生成V2
    - 添加规则辅助生成文章
    '''
    def __init__(self):
#         config = app_config
#         self.finance_du_host = config.get("finance_du_host")
        self.finance_du_host = "https://surreal.aidigger.com/api/v1/du/finance"
#         self.version = config.get('version')
        
        self.labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
        self.labelmap = {
                         '导语': ['导语','意图_企业基本信息介绍_企业基本信息介绍', '意图_企业活动_公司公告信息', '意图_企业业绩_现状偏好', '意图_企业业绩_现状偏坏'],
                         '业绩变动原因': ['意图_企业业绩_业绩变动原因'],
                         '主营业务$业务进展': ['意图_企业业务_业务介绍', '意图_企业业务_业务规模', '意图_企业业务_业务规划'],
                         '未来计划': ['意图_企业业务_业务前景', '意图_企业业绩_前景偏好','意图_企业业绩_前景偏坏'],
                         'other': ['其他', '意图_评论_风险提示','意图_评论_投资建议', '意图_评论_盈利预测', '意图_企业活动_人事活动', '意图_企业活动_融资活动','意图_企业活动_投资活动']
                        }
        self.labelmapR = {}
        for key, value in self.labelmap.items():
            for v in value:
                self.labelmapR[v] = key
    
    def _get_features_from_du_host(self, docs):
        '''
        根据docs，返回其预测的类别及概率
        '''
        try:
            ret = requests.post(self.finance_du_host, json = docs, timeout = 30)
        except requests.ReadTimeout:
            print('Timeout')
            logger.warning("Time out when try to get finance docs features")
            return []
        if ret.status_code == 200:
            return ret.json()
        
    def get_features(self, docs):
        """
        20一组调用分类api
        """
        batch_size = 20
        batch = []
        results = []
        for doc in docs:
            batch.append(doc)
            if len(batch) == batch_size:
                ret = self._get_features_from_du_host(batch)
                if ret:
                    results.extend(ret)
                batch = []
        if len(batch) != 0:
            ret = self._get_features_from_du_host(batch)
            if ret:
                results.extend(ret)
        return results
    
    def _match(self, pattern, lst):
        for l in lst:
            for p in pattern:
                if p in l:
                    return True
        return False
    
    def _intentMap(self, predict):
        """
        将现有意图映射到芥末堆所给的意图上去, 并将权重求和
        """
        result = {}
        for l in self.labels:
            result[l] = 0
        for key in predict.keys():
            result[self.labelmapR[key]] += predict[key]
        return result       
    
    def _getParabyIntent(self, docs, itt, num, threshold = 0.5):
        """
        返回给定意图概率（>threshold）前num的文本，
        """
        intent = {x['content']: x['features']['intents'] for x in docs}
        for key, value in intent.items():
            intent[key] = {x['name']: x['prob'] for x in value}
            intent[key] = self._intentMap(intent[key])
        intentL = {}
        for l in self.labels:
            intentL[l] = {}
        for k, v in intent.items():
            for i, p in v.items():
                intentL[i][k] = p
        for i in intentL.keys():
            intentL[i] = sorted(intentL[i].items(), key=operator.itemgetter(1))
        return [p[0] for p in intentL[itt][-num:] if (p[1] > threshold)]
    
    def _getPara(self, match, style, paragraphs, filter_headline = True, crap = None):
        """
        给定条件，返回符合条件的段落。
        """
        paras_raw = [p for p in paragraphs if p['name'] == style 
                if p['parents'] if self._match(match, p['parents'])
                if not p['content'].startswith('公司是否需要')]
        paras = [p['content'] for p in paras_raw]
        if not crap is None:
            index = len(paras)
            indexcontent = ''
            flag = False
            for i in range(len(paras)):
                for parent in paras_raw[i]['parents']:
                    if crap in parent:
                        indexcontent = paras_raw[i]['content']
                        flag = True
                if flag is True:
                    break
            if indexcontent != '':
                index = paras.index(indexcontent)
            paras = paras[0:index]
            paras_raw = paras_raw[0:index]
        if not filter_headline:
            for i, p in enumerate(paras_raw):
                if not p['parents'][0] in paras:
                    index = paras.index(p['content'])
                    paras.insert(index, p['parents'][0])
        return paras

    def _filter_text(self, text):
        if '商业模式' in text and '未发生' in text:
            return True
        if '年度内变化统计' in text:
            return True
        if len(text) <= 15:
            return True
        if u'□' in text or u'√' in text:
            return True
        return False    
        
    def _get_reason(self, finance_type, style, paragraphs, mode = None):
        '''根据finance_type({'hushen', 'xinsanban'})和style({'season', 'year'}，目前只做year), 
        返回由规则及模型生成的 *业绩变动原因* 意图段落
        return type: list(str)
        '''
        #A股规则
        if finance_type == 'hushen':
            if style == 'year':
#                 paras_raw = [{'content': p} for p in self._getPara(['经营情况讨论与分析'], 'Paragraph', paragraphs)]
#                 headline_raw = [{'content': p} for p in self._getPara(['、概述'], 'Headline', paragraphs)]
#                 docs = self.get_features(paras_raw)
#                 paras = self._getParabyIntent(docs, '业绩变动原因', 2)
                paras = [p for p in self._getPara(['经营情况讨论与分析'], 'Paragraph', paragraphs)
                     if ('营业收入' in p or '净利润' in p) and 
                     ('原因' in p or '因为' in p or '因此' in p) if not self._filter_text(p)]
                return paras
            else:
                paras = [p['content'] for p in paragraphs if self._match(['变动的情况及原因'], p['parents']) 
                         if '营业收入' in p['content'] or '净利润' in p['content'] or '主营业务收入' in p['content']]
                return paras
        #新三板规则
        else:
            paras = [p['content'] for p in paragraphs if p['name'] == 'Paragraph' 
                     if ('营业收入' in p['content'] or '净利润' in p['content']) and 
                     ('原因' in p['content'] or '因为' in p['content'] or '因此' in p['content'])]
            return paras
    
    def _get_mainB(self, finance_type, style, paragraphs, mode = None):
        if finance_type == 'hushen':
            if style == 'year':
                paras = self._getPara(['从事的主要业务'], 'Paragraph', paragraphs)
                return [paras[0]]
            else:
                return []
        else:
            paras = self._getPara(['商业模式'], 'Paragraph', paragraphs)
            paras = [p for p in paras if not self._filter_text(p)]
            return paras
    
    def _get_BProgress(self, finance_type, style, paragraphs, mode = None):
        if finance_type == 'hushen':
            if style == 'year':
                paras1_raw = [{'content': p} for p in self._getPara(['核心竞争力分析'], 'Paragraph', paragraphs)]
                paras1 = self._getParabyIntent(self.get_features(paras1_raw), '主营业务$业务进展', 2)
                paras2_raw = [{'content': p} for p in self._getPara(['、概述', '经营情况的讨论与分析'], 'Paragraph', paragraphs)]
                paras2 = self._getParabyIntent(self.get_features(paras2_raw), '主营业务$业务进展', 2)
                paras = paras1 + paras2
                return paras
            else:
                return []
        else:
            paras = self._getPara(['经营情况', '总体回顾'], 'Paragraph', paragraphs, filter_headline = False, crap = '主营业务分析')
            paras = [p for p in paras if not self._filter_text(p) if not '原因' in p or '因为' in p or '因此' in p]
            return paras
    
    def _get_future(self, finance_type, style, paragraphs, mode = None):
        if finance_type == 'hushen':
            if style == 'year':
                return self._getPara(['经营计划', '发展计划', '经营工作计划', '经营管理计划', '发展战略'], 'Paragraph', paragraphs)
            else:
                return []
        else:
            paras = self._getPara(['发展战略', '经营计划'], 'Paragraph', paragraphs)
            return paras
    
        
    def feature_process(self, item):
        '''处理财报pdf，解析为段落，标注意图
        Args:
            item: 原始财报pdf内容，应该包括
                - pdf_id: id
                - year: 年份
                - quarter: 季度
                - name：股票名称
                - code：股票代码
                - finance_type： hushen 和 xsb 两种类型
                - pubdate： 发布日期
                - url：财报链接
                - tables： 表格数据， json string
                - paragraphs： 段落数据，json string
        return：
            article：
                根据paragraphs生成的文章
        '''
        finance_type = item['finance_type']
        quarter = item['quarter']
        paragraphs = item['paragraphs']
        companyname = item['name']
        style = 'year'
        if quarter == 1 or quarter == 3:
            style = 'season'
        
        #获取业绩变动原因段落
        reason = self._get_reason(finance_type, style, paragraphs)
        #获取主营业务段落
        mainB = self._get_mainB(finance_type, style, paragraphs)
        #获取业务进展段落
        BProgress = self._get_BProgress(finance_type, style, paragraphs)
        #获取未来计划段落
        future = self._get_future(finance_type, style, paragraphs)
        
        article = reason + ['\n'] + mainB + ['\n'] +  BProgress + ['\n'] + future
        for i in range(len(article)):
            article[i] = article[i].replace('公司', companyname)
        return '\n'.join(article)
        


class FinancePreprocessV1(object):
    def __init__(self, app_config):
        config = app_config
        self.finance_parse_host = config.get("finance_parse_host")
        self.finance_du_host = config.get("finance_du_host")
        self.version = config.get("version")
        self.risk_words = ['风险提示','风险']

    def _get_features_from_du_host(self, docs):
        try:
            ret = requests.post(self.finance_du_host, json=docs, timeout=30)
        except requests.ReadTimeout:
            logger.warning("Time out when try to get finance docs features")
            return []
        if ret.status_code == 200:
            return ret.json()

    def get_features(self, docs):
        batch_size = 20
        batch = []
        results = []
        for doc in docs:
            batch.append(doc)
            if len(batch) == batch_size:
                ret = self._get_features_from_du_host(batch)
                if ret:
                    results.extend(ret)
                batch = []
        if len(batch) != 0:
            ret = self._get_features_from_du_host(batch)
            if ret:
                results.extend(ret)
        return results

    def parse_pdf(self, url):
        try:
            payload = {'url':url}
            ret = requests.get(self.finance_parse_host, params=payload, timeout=300)
        except:
            return []
        
        if ret.status_code == 200:
            return ret.json()

    def is_risk_paragraph(self, headline):
        headline = ' '.join(headline)
        for word in self.risk_words:
            if word in headline:
                return True
        return False

    def feature_process(self, item):
        '''处理财报pdf，解析为段落，标注意图
        Args:
            item: 原始财报pdf内容，应该包括
                - url: pdf的url
                - articleid
                - name
                - code
                - year
                - quarter
                - 其他
        '''
        url = item['url']
        paragraphs = self.parse_pdf(url)

        if not paragraphs:
            return None

        docs_raw = [{'content':p['content']} for p in paragraphs]
        headlines_raw = [{'content':p['headline'][-1] if p['headline'] else ''} for p in paragraphs]

        docs = self.get_features(docs_raw)
        headlines = self.get_features(headlines_raw)

        for i,hd in enumerate(headlines):
            docs[i]['title'] = ' ## '.join(paragraphs[i]['headline'])
            docs[i]['seq'] = paragraphs[i]['seq']
            docs[i]['report_type'] = 'financial'
            risk = self.is_risk_paragraph(paragraphs[i]['headline'])

            for j,intent in enumerate(hd['features']['intents']):
                if not risk:
                    docs[i]['features']['intents'][j]['prob'] += intent['prob']
                else:
                    docs[i]['features']['intents'][j]['prob'] = 0.0

            # risk paragraphs
            if risk:
                 docs[i]['features']['intents'][15]['prob'] = 1.0

            docs[i]['article'] = item['articleid']
            docs[i]['id'] = '{0}_{1}'.format(item['articleid'],i)

        # remove paragraph that is belong to '其他' intent
        docs = [doc for doc in docs if doc['features']['intents'][14]['prob'] < 1.0]

        # append other attribute back to doc
        keys = [key for key in item.keys() if key not in [
            'articleid', 'content']]
        for doc in docs:
            doc.update({k: item[k] for k in keys})
            # add version
            doc.update({"model_version":self.version})
        return docs

class FinancePreprocess(object):
    def __init__(self, app_config):
        config = app_config
        self.finance_du_host = config.get("finance_du_host")
        self.version = config.get("version")

        self.stock_types = ['hushen', 'xinsanban']

        # load stock info data
        stock_info_oss = config.get("stock_info_oss")
        stock_info_local = utils.oss_to_local(stock_info_oss, "/tmp")
        stock_info = pickle.load(open(stock_info_local, 'rb'))
        self.stock_name2id = stock_info['n2i']
        self.stock_name2type = stock_info['n2t']
        self.stock_kw_classifier = KeyWordClassifier(
            weighted=False, keywords=self.stock_name2id.keys())

        key_words = ['发布', '营业收入', '营收', '归属于上市公司股东的净利润',
                     '营业利润', '归属母公司净利润', '净利润', '归母净利润', '同比', '财务报告']
        quarter_keys = [
            ['q1', '第一季度', '一季度', '一季报'],
            ['q2', '半年度', '半年报'],
            ['q3', '第三季度', '三季度', '三季报'],
            ['q4', '年度报告', '年报']
        ]

        self.content_kw_classifier = KeyWordClassifier(
            weighted=False, keywords=key_words)
        self.quarter_kw_classifiers = [KeyWordClassifier(
            weighted=False, keywords=kws) for kws in quarter_keys]

    def _get_features_from_du_host(self, docs):
        try:
            ret = requests.post(self.finance_du_host, json=docs, timeout=10)
        except requests.ReadTimeout:
            logger.warning("Time out when try to get finance docs features")
            return []
        if ret.status_code == 200:
            return ret.json()

    def get_features(self, docs):
        batch_size = 20
        batch = []
        results = []
        for doc in docs:
            batch.append(doc)
            if len(batch) == batch_size:
                ret = self._get_features_from_du_host(batch)
                if ret:
                    results.extend(ret)
                batch = []
        if len(batch) != 0:
            ret = self._get_features_from_du_host(batch)
            if ret:
                results.extend(ret)
        return results

    def feature_process(self, item):
        '''处理原始的文章，将其划分为段落，并进行意图分类

        Args:
            item: 包含原始文章的所有信息的dict,要求必须包括的内容有：
                - articleid: 文章id
                - content: 文本内容
                - pubdate: 发布日期
                - title: 文章标题
                - source: 文章来源，要求为以下四种：
                    - hushen: A股
                    - xinsanban: 新三板
                    - jiemodui*: 芥末堆研报
                    - tonghuashun**: 同花顺研报
                    - 未来其他的源
                - name: 股票名称
                - code: 股票代码
                - year: 年份
                - quarter: 季度
            *如果文章类型为jiemodui，不需要包括股票名称在内之后的信息，会自动解析判断
            **如果文章类型为tonghuashun，不需要年份、季度信息，会自动解析判断

        Return:
            如果输入是财报或者能够解析出年份季度的研报，将会返回docs。否则返回None，无需处理。
            docs: 分段后的结果，是一个list of doc，其中每个doc包含以下这些key:
                - id: 段落唯一标识
                - content: 段落内容
                - title: 段落标题
                - pubdate: 发布日期
                - seq: 段落在文章中的位置
                - source: 文章来源，同输入
                - report_type: 文章类型，包括以下两种：
                    - financial: 公司发布的财报
                    - research: 研报，如同花顺上的研报，芥末堆的研报
                - name: 股票名称
                - code: 股票代码
                - year: 财报年份
                - quarter: 财报季度
                - model_version: 模型版本号
                - article: 文章id
                - features: 段落特征，只需要插入到SQL表格中，无需插入ES
        '''
        articleid = item['articleid']
        source_type = item['source']
        content = item['content']
        pubdate = item['pubdate']
        title = item['title']

        stock_name = item.get('name', None)
        stock_id = item.get('code', None)
        year = int(item.get('year', 0))
        quarter = int(item.get('quarter', 0))

        # split the content into paragraphs
        if source_type in self.stock_types:
            # do nothing here
            pass
        elif source_type == 'tonghuashun':
            ret = self.get_stock_info_from_research(
                stock_name, stock_id, title, content, pubdate)
            if ret:
                stock_id, stock_name, year, quarter = ret
            else:
                return None

        elif source_type == 'jiemodui':
            ret = self.get_stock_info_from_jmd(title, content, pubdate)
            if ret:
                stock_id, stock_name, year, quarter = ret
            else:
                return None
        else:
            raise ValueError(
                "source type of {0} is not supported yet.".format(source_type))

        logger.info(u"start to parse article {0} with source type {1}, stock name {2}, year {3}, quarter {4}, version {5}".format(
            articleid, source_type, stock_name, year, quarter, self.version))
        docs = parse_report(articleid, content, source_type,
                            stock_name, stock_id, year, quarter, pubdate, self.version)
        if docs:
            docs = self.get_features(docs)
            return docs
        else:
            return None

    def predict_year_quarter(self, pubdate):
        '''根据文章发布日期预测大致的年份和季度
        '''
        # in case pubdate is YYYY-MM-DDTHH:MM:SS
        pubdate = pubdate.split("T")[0]
        year, month, day = map(int, pubdate.split("-"))

        # 一般公司的报告会在季度之后才会发布，因此预测季度应该为当前发布日期季度减1
        quarter = (month - 1) // 3
        # 上一年年报
        if quarter == 0:
            quarter = 4
            year -= 1
        return year, quarter

    def predict_from_content(self, content, pubdate):
        '''根据文章内容进一步判断年份和季度
        '''
        # get the predicted year and quarter
        year, quarter = self.predict_year_quarter(pubdate)

        # only use the first two paragraphs to predict
        content = "".join(content.split("\n")[:2])

        predict, _ = self.content_kw_classifier.predict(content, method=1)
        # no key words found in the content
        if predict < 1.0:
            return None

        # if we found quarter key words in the content, return the year and quarter
        for i in range(4):
            predict_quarter, _ = self.quarter_kw_classifiers[i].predict(
                content, method=1)
            if predict_quarter >= 1.0:
                # predicted quarter is later than pubdate quarter
                # this should be last year's report
                if quarter < i + 1:
                    return year - 1, i + 1
                return year, i + 1

        # if we found at least 2 key words in the content, return the predicted year and quarter
        # else we can not tell if this is really a article about finance report
        if predict >= 2.0:
            return year, quarter
        else:
            return None

    # pubdate shoule be in format YYYY-MM-DD
    def get_stock_info_from_jmd(self, title, content, pubdate):
        """解析芥末堆数据
        """
        title = title.replace(" ", "")
        # 只采用财报季文章
        if u"【财报季】" not in title:
            return None

        _, stocks = self.stock_kw_classifier.predict(title, method=1)
        # 不处理没有股票，或者存在多个股票的情况
        if len(stocks) != 1:
            return None

        ret = self.predict_from_content(content, pubdate)

        if ret:
            year = ret[0]
            quarter = ret[1]
            stock_name = stocks[0]
            stock_id = self.stock_name2id[stock_name]
            return stock_id, stock_name, year, quarter
        return None

    def get_stock_info_from_research(self, stock_name, stock_id, title, content, pubdate):
        '''解析研报获得股票年份信息
        '''
        ret = self.predict_from_content(content, pubdate)
        if ret:
            year = ret[0]
            quarter = ret[1]
            return stock_id, stock_name, year, quarter
        return None

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-30 14:28:52.902394. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [289]:
fin_A_bnb = FinancePreprocessV2()
print(items[3]['url'])
print(fin_A_bnb.feature_process(items[3]))

http://disclosure.szse.cn/finalpage/2017-04-26/1203387778.PDF
1、本期主营业务收入为30,593.59万元，上年同期为18,638.03万元，增长64.15%，主要是因为立思辰主








/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-30 14:28:59.589965. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [286]:
paras = [p['content'] for p in items[3]['paragraphs'] if fin_A_bnb._match(['变动的情况及原因'], p['parents']) 
         if '营业收入' in p['content'] or '净利润' in p['content'] or '主营业务收入' in p['content']]
# para = [p['content'] for p in items[3]['paragraphs'] if '本期主营业务收入']
paras

['1、本期主营业务收入为30,593.59万元，上年同期为18,638.03万元，增长64.15%，主要是因为公司主']

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-30 14:23:46.745563. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [295]:
fin_A_bnb = FinancePreprocessV2()
print(items[0]['url'])
text0 = '''立思辰主营业务分教育与信息安全两大业务，教育业务主要产品包括面向 B 端用户的区域教育云平台、智慧校园整体解决方案、K12 领域的学科应用产品等以及面向 C 端用户的高考升学咨询服务、留学咨询服务、线上辅导等；信息安全业务主要为客户提供围绕数据全生命周期的数据安全解决方案、工控安全产品及解决方案、面向多行业的自主可控产品及解决方案等。'''
text0 in fin_A_bnb.feature_process(items[0])

http://disclosure.szse.cn/finalpage/2017-08-29/1203888889.PDF


True

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-30 14:35:16.932386. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [299]:
fin_A_nb = FinancePreprocessV2()
print(fin_A_nb.feature_process(items[2]))



立思辰主营业务分教育与信息安全两大业务，教育业务主要产品包括 K12 领域的学科应用产品、区域教育云平台、智慧校园整体解决方案、高考升学咨询服务、留学服务以及就业创业学分教育解决方案等；信息安全业务主要为客户提供围绕数据全生命周期的数据安全解决方案、工控安全产品及解决方案、面向多行业的自主可控产品及解决方案等。


课程是连接学生和学习的桥梁，在教学中老师重视调动学生的动手能力，让他们去大胆创作，最大限度地发挥学生的主动性、创作性、发展性，从而达到激发兴趣、启迪心智、感悟积淀的三重境界。立思辰将开设更多校本课程，从科学技术，生活艺术，传统文化等方面提取素材，为学生提供更多选择。
康邦校本课程解决方案分为科学技术、生活艺术、传统文化、项目主题四大类别，包含课程、师资、场景、活动四个维度，以权威名师为支撑，配备专业教材，指导手册，精品课程资源、课件、教案，并制定有针对性的教学方法和课程评价。康邦精品校本课程将结合学校的校园文化和特色课程开发，进行师资培训、师资输出、场景建设以及活动策划等服务，全方位保证精品校本课程的开设，为学校一线教学提供有力支持。
①智慧教育
乐易考是大学生就创业教育及服务的专业品牌，主要为高校提供就业创业学分教育解决方案，包括教材，双创MOOC平台与课程，双创线下翻转课堂以及相关的师资培训。目前，乐易考与湖南省厅共建的湖南就创业网络学院已经签约12所高校。


立思辰是一家新生态的教育集团，秉承“以科技和人文改变教育”的历史使命，坚持“做大智慧教育，做强教育服务，做精未来学校，做实教育生态”的发展路径，利用科技手段打通校内校外、线上线下、国内国外的学习场景，让学生们高效学习、快乐成长，最终实现“激发成就亿万青少年”的宏伟愿景。
1.做大智慧教育。智慧教育是立思辰教育业务的基石。以立思辰康邦为核心，发挥领先者优势，用科技手段构建教育信息环境，卡位和布局学校教育场景，持续扩大业务规模。
2.做强教育服务。教育服务是立思辰教育战略的关键和重点，围绕“学习-升学”的核心诉求，提供针对公立校的教学服务解决方案（包括但不仅限于学科应用、教学评价、教师培训、专业建设等等）、针对C端市场的学习服务解决方案（包括但不仅限于学习辅导、海内外升学指导等）。通过科技打通校内校外、线上线下、国内国外，成为未来全球领先的教育服务提供商。
3.做精未来学校。通过教育产业基金

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-30 14:37:00.290256. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [300]:
fin_xsb_bnb = FinancePreprocessV2()
print(items[4]['url'])
print(fin_xsb_bnb.feature_process(items[4]))

http://www.cninfo.com.cn/finalpage/2017-08-21/1203841042.PDF
（1）报告期，营业收入增长的主要原因是：报告期内行业政策进一步开放，世纪明德国内研学、国际
游学及教师培训等主营业务均稳步增长；上半年世纪明德继续加强游学课程研发，增加了课程数量，丰富了
游学课程内容，提升了服务品质，同时通过品牌的大力推广及市场开发方面的大力投入，使得营业收入
继续稳定增长。
（3）报告期内，世纪明德净利润增长的原因是：2017  年上半年经营收入的稳定增长，世纪明德对供应链的
不断整合以及内部控制的加强，期间费用的年均增长速度低于营业毛利的年均增长速度，使得营业利润
增长了 40.03%；日常资金的有效管理，使得投资收益增增长率高达 108.95%。
2）经营活动产生的现金流量净额同比增长  60,927,325.28  元，较上年同期增加的主要原因是:  报告
期内，世纪明德丰富产品种类、大力发展销售业务，世纪明德业务收入较上年同期增长  48.52%，客户数量大幅
增长，世纪明德营业收入增加，导致经营活动产生的现金流量净额大幅增长。


世纪明德为游学服务商龙头企业，为 3 至 18 岁青少年提供国内研学、国际游学、夏冬令营服务；为广
大教育工作者提供教师培训、教育论坛服务。业务涵盖游学产品研发、销售推广、业务接待和后续服务
全套流程。
世纪明德的研学产品，将青少年的成长和收获置于产品价值的首要位置。报告期内，世纪明德继续加大产品
课程设计的投入，以研学主题、研学课程、研学导师、研学基地、研学线路、安全管理六大要素为基础，
以学生为中心，以课程为核心，以接待为关键，寓教于游，用一种知识整合的形式将各种学科内容的传
授和实践能力的培养融入到研学活动之中。
世纪明德的教师培训业务主要为教育工作者提供学习交流论坛。通过了解客户对于教育培训方面的需
求，为客户定制一整套培训方案，包括设计培训内容、邀请知名教育专家，安排培训场地、交通食宿等。
世纪明德为客户提供的教育培训是基于世纪明德在教育领域的长期运作经验，常年积累的教育领域专家、学者资
源。报告期内，中国教育明德论坛相继在各地举办，取得了良好的社会效益和经济效益。
得益于世纪明德快速的研发能力，世纪明德的服务流程进一步升级。基于完善的整体化、综合化和数据化的
信息管理系统，各部

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-30 14:39:00.265181. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [298]:
fin_xsb_nb = FinancePreprocessV2()
docs = fin_xsb_nb.feature_process(items[1])
text1 = '''1.营业收入：报告期较上年同期增长 56.52%，主要原因系：（1）市场环境向好：国家研学旅行政策
不断落地，政策环境向好，市场需求大增，客户人数增长迅速；（2）产品线丰富：报告期间世纪明德加强了国
内研学、国际游学、社会实践以及教师培训产品的研发，产品可以覆盖更大客户群，可以为存量客户提供
更多的产品服务内容；（3）市场开拓方面，世纪明德制定积极的销售政策，重点突破市场薄弱区域，客户范围
不断扩大。重点突破春秋季研学旅行市场，从客户参营时间上进行延展，从一年中寒暑假的游学旺季拓展
为全年旺季。'''
print(text1 in docs)

True


/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-30 14:36:01.180362. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [ ]:
item = items[1]['paragraphs']
[i for i in item if i['name'] == 'Paragraph' if fin._match(['总体回顾'], i['parents'])]

In [279]:
print(items[3]['url'])
print(items[3]['pdf_id'])
[p['content'] for p in items[3]['paragraphs'] if p[]]

http://disclosure.szse.cn/finalpage/2017-04-26/1203387778.PDF
300010_2017_1


/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-30 14:14:09.717339. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [103]:
import json
itemlist = ['000001_0', '000004_0', '000006_0']
keys = ['pdf_id', 'year', 'quarter', 'name', 'code', 'finance_type', 'pubdate', 'url', 'tables', 'paragraphs']
items = []
for item in itemlist:
    f = open('/data/xueyou/finance/pdf_content_test/' + item)
    for line in f:
        data = line.split('\x01')
        tmp = {}
        for i, key in enumerate(keys):
            try:
                js = json.loads(data[i])
                tmp[key] = js
            except:
                tmp[key] = data[i]
        items.append(tmp)
    f.close()

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-27 11:31:08.361913. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [277]:
with open('/data/share/items.pkl', 'wb') as pickle_file:
    pickle.dump(items, pickle_file)

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-29 14:56:50.715807. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [278]:
f = open('/data/share/items.pkl', 'rb')
import pickle
items_ = pickle.load(f)
items_[0].keys()

dict_keys(['quarter', 'url', 'year', 'finance_type', 'pubdate', 'name', 'code', 'pdf_id', 'paragraphs', 'tables'])

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-29 14:57:50.969544. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [37]:
labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
labelmap = {
                 '导语': ['导语','意图_企业基本信息介绍_企业基本信息介绍', '意图_企业活动_公司公告信息', '意图_企业业绩_现状偏好', '意图_企业业绩_现状偏坏'],
                 '业绩变动原因': ['意图_企业业绩_业绩变动原因'],
                 '主营业务$业务进展': ['意图_企业业务_业务介绍', '意图_企业业务_业务规模', '意图_企业业务_业务规划'],
                 '未来计划': ['意图_企业业务_业务前景', '意图_企业业绩_前景偏好','意图_企业业绩_前景偏坏'],
                 'other': ['其他', '意图_评论_风险提示','意图_评论_投资建议', '意图_评论_盈利预测', '意图_企业活动_人事活动', '意图_企业活动_融资活动','意图_企业活动_投资活动']
                }
labelmapR = {}
for key, value in labelmap.items():
    for v in value:
        labelmapR[v] = key

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-26 17:23:56.036668. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [269]:
import operator
def intentMap(predict):
        """
        将现有意图映射到芥末堆所给的意图上去, 并将权重求和
        """
        result = {}
        for l in labels:
            result[l] = 0
        for key in predict.keys():
            result[labelmapR[key]] += predict[key]
        return result       
    
def getParabyIntent(docs, itt, num, threshold = 0.5):
    intent = {x['content']: x['features']['intents'] for x in docs}
    for key, value in intent.items():
        intent[key] = {x['name']: x['prob'] for x in value}
        intent[key] = intentMap(intent[key])
    intentL = {}
    for l in labels:
        intentL[l] = {}
    for k, v in intent.items():
        for i, p in v.items():
            intentL[i][k] = p
    for i in intentL.keys():
        intentL[i] = sorted(intentL[i].items(), key=operator.itemgetter(1))
    return [p[0] for p in intentL[itt][-num:] if (p[1] > threshold)]
import requests
text = {'content':'''立思辰教育秉承“以科技和人文改变教育”的历史使命，坚持“做大智慧教育，做强教育服务，做精未来
学校，做实教育生态”的发展路径，利用科技手段打通校内校外、线上线下、国内国外的学习场景，让学生
们高效学习、快乐成长，最终实现“激发成就亿万青少年”的宏伟愿景。

'''}
query = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
print(getParabyIntent(query, '主营业务$业务进展', 2))

['立思辰教育秉承“以科技和人文改变教育”的历史使命，坚持“做大智慧教育，做强教育服务，做精未来\n学校，做实教育生态”的发展路径，利用科技手段打通校内校外、线上线下、国内国外的学习场景，让学生\n们高效学习、快乐成长，最终实现“激发成就亿万青少年”的宏伟愿景。\n\n']


/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-29 11:21:58.526832. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [184]:
a = '公司主营业务分教育与信息安全两大业务，教育业务主要产品包括 K12 领域的学科应用产品、区域教育云平台、智慧校园整体解决方案、高考升学咨询服务、留学服务以及就业创业学分教育解决方案等；信息安全业务主要为客户提供围绕数据全生命周期的数据安全解决方案、工控安全产品及解决方案、面向多行业的自主可控产品及解决方案等。'
a.replace('公司', '立思辰')
a

'公司主营业务分教育与信息安全两大业务，教育业务主要产品包括 K12 领域的学科应用产品、区域教育云平台、智慧校园整体解决方案、高考升学咨询服务、留学服务以及就业创业学分教育解决方案等；信息安全业务主要为客户提供围绕数据全生命周期的数据安全解决方案、工控安全产品及解决方案、面向多行业的自主可控产品及解决方案等。'

/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-27 16:05:28.728488. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [224]:
print( items[1]['url'])

http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF


/opt/anaconda/anaconda3/lib/python3.5/site-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2018-01-29 10:36:18.077282. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [ ]:
paragraphs = items[1]['paragraphs']
paras_raw = [p for p in paragraphs if p['name'] == 'Paragraph' 
                if p['parents'] if fin_xsb_bnb._match(['总体回顾'], p['parents'])
                if not p['content'].startswith('公司是否需要')]
paras = [p['content'] for p in paras_raw]

index = len(paras)
indexcontent = ''
flag = False
for i in range(len(paras)):
    for parent in paras_raw[i]['parents']:
        if '主营业务分析' in parent:
            indexcontent = paras_raw[i]['content']
            flag = True
    if flag is True:
        break
if indexcontent != '':
    index = paras.index(indexcontent)
paras = paras[0:index]
paras_raw = paras_raw[0:index]
print(paras_raw[:5])
for p in paras:
    print(p)
    print ('-')